In [ ]:
# @title
################################################
# Staging core library and .CSV files
################################################

from os import chdir
from pathlib import Path
import subprocess

link_prefix = "https://github.com/alinnman/celestial-navigation/raw/refs/heads/main/"
data_prefix = "sample_data/"

for t in ["starfix", "notebook_helper"]:
    s = t + ".py"
    my_file = Path (s)
    if not my_file.exists():
        print ("Fetching " + s + ".")
        subprocess.run (["wget", link_prefix + s])
    else:
        print ("Python library " + s + " exists.")

try:
    chdir (data_prefix)

    for t in ["planets","stars","sun-moon-sd","sun-moon","venus-mars-hp"]:
        s = t + ".csv"
        my_file = Path(s)
        if not my_file.exists():
            print ("Fetching " + s + ".")
            subprocess.run (["wget", link_prefix + data_prefix + s])
        else:
            print ("Machine-readable almanac data " + s + " exists.")
finally:
    chdir ("..")

# Celestial Navigation Workbench

## Introduction

This is a sample web app for celestial navigation for a stationary observer.<br/>

Instruction:

1. **Press Ctrl+F9 to run the notebook**. <br/>
On a mobile phone or other keyboard-less device use the menu alternative "Runtime --> Run all" <br/>
You will generate an input form and a sight reduction based on default data. 
The bottom cell will show the coordinate and a map of the resulting intersections. Failure to perform a sight reduction will produce an error message.
The input form can be manipulated to enter new data. 
1. The entered parameters in the form below were taken for an observation taken by me on vacation this summer.

**NOTE**: When running the notebook for the first time you may get a security warning about the code not originating from Google Colab. You can safely ignore this warning and continue.

**NOTE**: The first execution of the notebook can take some time. This is due to Google Colab initialization.

This is part of (and a demo of) a software package, primarily for using on a mobile phone (without need for an internet connection). It is written in Python and also uses Jupyter/Colab features for web browser support. The target platform is the PyDroid 3 app for Android. For more information see [here](https://github.com/alinnman/celestial-navigation/tree/main/README.md).

For a short intro to the workflow and algorithm used see [here](https://github.com/alinnman/celestial-navigation/tree/main/WORKFLOW.md).

Geographical Positions of celestial objects (Declination, SHA, GHA) and parallax information (HP) are initialized from
a machine-readable nautical almanac residing in the "sample_data" data folder.
The data covers the years 2024-2028. The csv files are loaded when you startup the notebook.
(These values can also be entered by hand if you use the Python scripts).

A human-readable Nautical Almanac for 2024 can be found [here](https://github.com/alinnman/celestial-navigation/blob/main/nautical_almanacs/NAmod(A4)_2024.pdf).

A human-readable Nautical Almanac for 2025 can be found [here](https://github.com/alinnman/celestial-navigation/blob/main/nautical_almanacs/NAmod(A4)_2025.pdf).


## How to fill in the form

Format for all angles is "DD:MM:SS", "DD:MM" or "DD" (degrees, minutes, seconds). <br/>
Timestamps [use the ISO 8601 format](https://en.wikipedia.org/wiki/ISO_8601).<br/>

The **DRP_LAT** and **DRP_LON** fields represent your estimated position. Having a reasonably good estimate will improve the results of your sight reduction. 

The "Use #" check boxes are used to select the sight you want to use. Default is using three sights. You may only use 2 by unselecting one check box. 

* The **NAME_#** text fields are mandatory. You must enter a name of a celestial object.<br>
* The **ALT_#** text fields are mandatory. You enter the measured altitude of the celestial object above the horizon (using your sextant).<br>
* The **TIME_#** text fields are mandatory. You enter the time taken from your clock/chronometer.<br>
* The ARTIFICAL_HOR check box should be checked if you use an artifical horizon.<br>
* For description of the other fields see [here](https://github.com/alinnman/celestial-navigation/blob/main/README.md#parameters).

In [ ]:
import notebook_helper
from ipywidgets import VBox

notebook_helper.initialize ("notebook.1.json",
                {"ObjectName1" : "Sun",
                "Altitude1" : "55:8:1.1",
                "Time1" : "2024-05-05 15:55:18+00:00",
                "LimbCorrection1" : "0",
                "IndexError1" : "0",
                "ArtificialHorizon1" : "False",
                "ObserverHeight1" : "0",
                "Temperature1" : "10",
                "TemperatureGradient1" : "-0.01",
                "Pressure1" : "101",

                "ObjectName2" : "Sun",
                "Altitude2" : "19:28:19",
                "Time2" : "2024-05-05 23:01:19+00:00",
                "LimbCorrection2" : "0",
                "IndexError2" : "0",
                "ArtificialHorizon2" : "False",
                "ObserverHeight2" : "0",
                "Temperature2" : "10",
                "TemperatureGradient2" : "-0.01",
                "Pressure2" : "101",

                "ObjectName3" : "Vega",
                "Altitude3" : "30:16:23.7",
                "Time3" : "2024-05-06 04:04:13+00:00",
                "LimbCorrection3" : "0",
                "IndexError3" : "0",
                "ArtificialHorizon3" : "False",
                "ObserverHeight3" : "0",
                "Temperature3" : "10",
                "TemperatureGradient3" : "-0.01",
                "Pressure3" : "101",

                "DrpLat" : "40",
                "DrpLon" : "-90",

                "Use1" : "True",
                "Use2" : "True",
                "Use3" : "True"})

w = notebook_helper.render_widget(
                [["ObjectName",          "𝗡𝗔𝗠𝗘",          "MyTextWidget"],
                 ["Altitude",            "𝗔𝗟𝗧",            "MyTextWidget"],
                 ["Time",                "𝗧𝗜𝗠𝗘",           "MyTextWidget"],
                 ["IndexError",          "INDEX_ERROR",    "MyTextWidget"],
                 ["LimbCorrection",      "LIMB_CORR",      "MyLimbDropdown"],
                 ["ArtificialHorizon",   "ARTIFICIAL_HOR", "MyCheckboxWidget"],
                 ["ObserverHeight",      "OBS_HEIGHT",     "MyTextWidget"],
                 ["Temperature",         "TEMP",           "MyTextWidget"],
                 ["TemperatureGradient", "TEMP_GRADIENT",  "MyTextWidget"],
                 ["Pressure",            "PRESSURE",       "MyTextWidget"]], 3)
VBox(w)

In [ ]:
# SIGHT REDUCTION.
the_map = notebook_helper.sight_reduction ()
the_map